<a href="https://colab.research.google.com/github/crisbpadilla/DataScience-M2M-course-Datatalent/blob/main/Widget_withBokeh_excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import cividis
from bokeh.models import CrosshairTool, ColumnDataSource

import pandas as pd

output_notebook()

columns=['year','state','candidate','party','candidatevotes','totalvotes']
df = pd.read_csv('1976-2016-president.csv',usecols=columns,dtype={'year':str})
state_group= df.groupby('state')
year=df['year'].unique()

color=cividis(len(state_group))
p=figure(tooltips= '$name @x:@y%',x_range= year)

state_line={}

for i,states in enumerate(state_group.groups):

  data=state_group.get_group(states)
  party_data=data[data['party']=='democrat']
  x=party_data['year']
  percentage=party_data['candidatevotes']/party_data['totalvotes']*100

  source = ColumnDataSource(data={'x' : x, 'y':percentage})

  # line= p.line(x=party_data['year'],y=percentage, color=color[i], name= states)
  line= p.line(x='x',y='y', color=color[i], name= states,source=source)

  state_line[states]=line

p.add_tools(CrosshairTool())
show(p)

In [9]:
for line in state_line.values():
  line.visible= False
show(p)
print(state_line)

{'Alabama': GlyphRenderer(id='p4240', ...), 'Alaska': GlyphRenderer(id='p4252', ...), 'Arizona': GlyphRenderer(id='p4264', ...), 'Arkansas': GlyphRenderer(id='p4276', ...), 'California': GlyphRenderer(id='p4288', ...), 'Colorado': GlyphRenderer(id='p4300', ...), 'Connecticut': GlyphRenderer(id='p4312', ...), 'Delaware': GlyphRenderer(id='p4324', ...), 'District of Columbia': GlyphRenderer(id='p4336', ...), 'Florida': GlyphRenderer(id='p4348', ...), 'Georgia': GlyphRenderer(id='p4360', ...), 'Hawaii': GlyphRenderer(id='p4372', ...), 'Idaho': GlyphRenderer(id='p4384', ...), 'Illinois': GlyphRenderer(id='p4396', ...), 'Indiana': GlyphRenderer(id='p4408', ...), 'Iowa': GlyphRenderer(id='p4420', ...), 'Kansas': GlyphRenderer(id='p4432', ...), 'Kentucky': GlyphRenderer(id='p4444', ...), 'Louisiana': GlyphRenderer(id='p4456', ...), 'Maine': GlyphRenderer(id='p4468', ...), 'Maryland': GlyphRenderer(id='p4480', ...), 'Massachusetts': GlyphRenderer(id='p4492', ...), 'Michigan': GlyphRenderer(id=

In [10]:
from ipywidgets import interact,SelectMultiple,IntRangeSlider,FloatSlider

In [15]:
from bokeh.io import output_notebook, push_notebook


handle = show(p, notebook_handle=True)
default_value=['Hawaii','Washington']

select_line= SelectMultiple( options=state_line.keys(),
                            rows=10, description='Select state(s):',
                             value= default_value)

@interact(states=select_line)
def update(states):
  for state_line_name,state_line_fig in state_line.items():
    if state_line_name in states:
      state_line_fig.visible= True
    else:
      state_line_fig.visible= False
  push_notebook(handle=handle)

#set up the range slider

#first create the object that contain the atributes of the range slider
min_year=int(year[0])
max_year=int(year[-1])

select_range= IntRangeSlider(value= [min_year,max_year],
                             min=min_year,max=max_year,step=4,
                             description='Year range')

@interact(years=select_range)#This is a tupple value ex:(california,colorado)
def update(years):
    x_years = range(years[0], years[1] + 1, 4)
    p.x_range.factors = list(map(str, x_years))#factors is how bokeh represents its data
    push_notebook(handle=handle)#rerender the plot with the changes
                                #that we had made

# 3rd widget float slider vertical to show just the lines that have certain % of
# votes for democraths

ptg= FloatSlider(value=100,min=0,max=100,step=5,description='Ptg votes:',disabled=False,
                 continuous_update=False,orientation='horizontal',readout=True,
                 readout_format='.1f')
@interact(condition= ptg)
def update(condition):
  for state_line_name,state_line_fig in state_line.items():
    if state_line_fig.visible is True:
      if max(state_line_fig.data_source.data['y']) > condition:
        state_line_fig.visible= False

  push_notebook(handle=handle)
  print(condition)





interactive(children=(SelectMultiple(description='Select state(s):', index=(11, 47), options=('Alabama', 'Alas…

interactive(children=(IntRangeSlider(value=(1976, 2016), description='Year range', max=2016, min=1976, step=4)…

interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Ptg votes:', readout_form…

In [13]:
# print(state_line)
# print(year[-1])
print(max(source.data['y']))


39.809265525159425
